<a href="https://colab.research.google.com/github/platypus2000jp/Colaboratory/blob/main/%E7%B5%B1%E8%A8%88%E7%9A%84%E5%9B%A0%E6%9E%9C%E6%8E%A8%E8%AB%96.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# prompt: サンプルデータに対して、因果フォレスト（Causal Forests）による統計的因果推論を示してください。

# 必要なライブラリのインストール
!pip install econml

# ライブラリのインポート
import numpy as np
import pandas as pd
from econml.dml import CausalForestDML
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

# サンプルデータの作成
np.random.seed(123)
n_samples = 1000
X = np.random.normal(size=(n_samples, 3))  # 説明変数
treatment = np.random.binomial(1, 0.5, size=n_samples)  # 介入変数
y = 2 * X[:, 0] + 3 * treatment + np.random.normal(size=n_samples)  # 目的変数

# データフレームに変換
data = pd.DataFrame({'y': y, 'treatment': treatment, 'X1': X[:, 0], 'X2': X[:, 1], 'X3': X[:, 2]})

# データを訓練データとテストデータに分割
X_train, X_test, y_train, y_test, treatment_train, treatment_test = train_test_split(
    data[['X1', 'X2', 'X3']], data['y'], data['treatment'], test_size=0.2, random_state=42
)

# Causal Forestモデルの作成
model = CausalForestDML(
    model_y=RandomForestRegressor(),
    model_t=RandomForestRegressor(),
    discrete_treatment=True
)

# モデルの訓練
model.fit(Y=y_train, T=treatment_train, X=X_train)

# 介入効果の推定
treatment_effect = model.effect(X_test)

# 結果の表示
print("平均介入効果:", np.mean(treatment_effect))


/usr/local/lib/python3.10/dist-packages/econml/dml/dml.py:58: UserWarning:

First stage model has discrete target but model is not a classifier!

/usr/local/lib/python3.10/dist-packages/econml/dml/dml.py:58: UserWarning:

First stage model has discrete target but model is not a classifier!



平均介入効果: 3.002155883548684


In [ ]:
# prompt: 上記の分析を作図できますか。plotlyでお願いします。

import plotly.express as px

# 介入効果のヒストグラムを作成
fig = px.histogram(treatment_effect, nbins=30, title="介入効果の分布")
fig.show()

# 介入効果と説明変数の散布図を作成（例としてX1とX2を使用）
fig = px.scatter(x=X_test['X1'], y=treatment_effect, color=X_test['X2'],
                 labels={'x': 'X1', 'y': '介入効果', 'color': 'X2'},
                 title="介入効果と説明変数の関係")
fig.show()


In [ ]:
# prompt: 介入効果と説明変数の関係についてカーネル回帰の平滑化も加えてください。

import numpy as np
import pandas as pd
from econml.dml import CausalForestDML
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import plotly.express as px
from sklearn.kernel_ridge import KernelRidge

# 必要なライブラリのインストール
!pip install econml

# ライブラリのインポート

# サンプルデータの作成
np.random.seed(123)
n_samples = 1000
X = np.random.normal(size=(n_samples, 3))  # 説明変数
treatment = np.random.binomial(1, 0.5, size=n_samples)  # 介入変数
y = 2 * X[:, 0] + 3 * treatment + np.random.normal(size=n_samples)  # 目的変数

# データフレームに変換
data = pd.DataFrame({'y': y, 'treatment': treatment, 'X1': X[:, 0], 'X2': X[:, 1], 'X3': X[:, 2]})

# データを訓練データとテストデータに分割
X_train, X_test, y_train, y_test, treatment_train, treatment_test = train_test_split(
    data[['X1', 'X2', 'X3']], data['y'], data['treatment'], test_size=0.2, random_state=42
)

# Causal Forestモデルの作成
model = CausalForestDML(
    model_y=RandomForestRegressor(),
    model_t=RandomForestRegressor(),
    discrete_treatment=True
)

# モデルの訓練
model.fit(Y=y_train, T=treatment_train, X=X_train)

# 介入効果の推定
treatment_effect = model.effect(X_test)

# 結果の表示
print("平均介入効果:", np.mean(treatment_effect))



# 介入効果のヒストグラムを作成
fig = px.histogram(treatment_effect, nbins=30, title="介入効果の分布")
fig.show()

# 介入効果と説明変数の散布図を作成（例としてX1とX2を使用）
fig = px.scatter(x=X_test['X1'], y=treatment_effect, color=X_test['X2'],
                 labels={'x': 'X1', 'y': '介入効果', 'color': 'X2'},
                 title="介入効果と説明変数の関係")

# カーネル回帰による平滑化
kr = KernelRidge(kernel='rbf', gamma=0.1)  # カーネルとハイパーパラメータを調整
kr.fit(X_test[['X1']], treatment_effect)
y_pred = kr.predict(X_test[['X1']])
fig.add_scatter(x=X_test['X1'], y=y_pred, mode='lines', name='Kernel Regression')

fig.show()


/usr/local/lib/python3.10/dist-packages/econml/dml/dml.py:58: UserWarning:

First stage model has discrete target but model is not a classifier!

/usr/local/lib/python3.10/dist-packages/econml/dml/dml.py:58: UserWarning:

First stage model has discrete target but model is not a classifier!



平均介入効果: 3.002155883548684


In [ ]:
# prompt: 因果効果であるCATE（Conditional Average Treatment Effect）を推定を表にまとめてください。plotlyを使用してください。

import pandas as pd
import plotly.graph_objects as go

# CATEの推定値をデータフレームに格納
cate_df = pd.DataFrame({'X1': X_test['X1'], 'CATE': treatment_effect})

# 表を作成
fig = go.Figure(data=[go.Table(
    header=dict(values=['X1', 'CATE']),
    cells=dict(values=[cate_df.X1, cate_df.CATE]))
])

fig.update_layout(title='CATE (Conditional Average Treatment Effect)')
fig.show()
